# Fraud Detection Project

## Collaboration Plan
**Participants:** Ryan Tang and Santiago von Straussburg

**Github Page**: https://ryantangmj.github.io

### Overview
We are collaboratively analyzing two fraud datasets to explore fraud patterns, feature importance, and machine learning model evaluation.

- **First dataset:** `Cargo_fraud_only.csv`, obtained from [UCR Database](https://cde.ucr.cjis.gov/LATEST/webapp/#/pages/downloads)
- **Second dataset:** Twelve-month and one-month arrest data for fraud, obtained from [UCR Database](https://cde.ucr.cjis.gov/LATEST/webapp/#/pages/downloads)
- **Third dataset:** Yearly unemployment index by State, obtained from [BLS Database](https://data.bls.gov/lausmap/showMap.jsp)


### Technologies Used
1. **GitHub Repository**: For version control, code collaboration, and final project hosting.
2. **Google Colab/Jupyter Notebooks**: For ETL, EDA, and model development.
3. **Discord**: Primary communication platform for real-time discussions.
   - Weekly meetings at 08:00 PM CST on Thursdays for progress reviews and planning.

### Current Focus
Both team members are currently working together on data exploration, including Extraction, Transformation, and Load (ETL) processes, as well as Exploratory Data Analysis (EDA).

### Roadmap & Milestones
#### Milestone 1 – Initial Dataset Selection & ETL
- Identify datasets. - **Completed**
- Perform initial ETL on datasets. - **Completed**
- Establish a GitHub repository and GitHub Pages site. - **Completed**
- Develop basic statistics and initial graph for dataset understanding. - **Completed**

#### Milestone 2 – Additional ETL & Exploratory Data Analysis
- Continue data cleaning and transformation. - **Completed**
- Conduct comprehensive EDA with 3-5 key graphs. - **Completed**
- Present the first project pitch with initial findings. - **Completed**

#### Deliverable 1 – In-Class Presentation
- Finalize and present a 5-7 slide deck covering problem statement, ETL, EDA, and project progress. - **Completed**

#### Milestone 3 – Model Development & Evaluation
- Select machine learning models (Random Forest, XGBoost, Logistic Regression).
- Begin model training and evaluation.
- Analyze model performance and feature importance.

#### Deliverable 2 – Final Website & Presentation
- Finalize project with the deployment of results to the GitHub Pages site.
- Prepare the final presentation summarizing the project lifecycle.

---

## Project Goals

The goal of this collaborative project is to analyze fraud patterns, identify significant features contributing to fraud, and evaluate various machine learning models for fraud detection. By leveraging two distinct datasets, we aim to develop a deep understanding of fraudulent behavior and build predictive models that will aid in identifying and mitigating fraud across different sectors. Specifically, our objectives are as follows:

### 1. Fraud Pattern Analysis
- **Objective:** Investigate patterns and trends in fraud activities across different sectors, particularly cargo-related fraud and arrest data for fraud cases. This will involve examining how fraudulent activities vary over time and geographic locations, and identifying key factors that influence fraud prevalence.

### 2. Feature Importance Assessment
- **Objective:** Analyze and rank the importance of different features contributing to fraud detection. By evaluating features such as transaction details, timestamps, geographic data, and socio-economic indicators, we aim to pinpoint the key variables that can most accurately predict fraud occurrences.

### 3. Machine Learning Model Development & Evaluation
- **Objective:** Develop and compare multiple machine learning models (Random Forest, XGBoost, Logistic Regression) to identify the most effective model for predicting fraud cases. The models will be evaluated on their performance using metrics such as accuracy, precision, recall, and F1-score to ensure robust fraud detection capabilities.

### 4. Comprehensive Data Analysis
- **Objective:** Through thorough data exploration and analysis, we aim to create a holistic view of fraud activities, utilizing exploratory data analysis (EDA) techniques. This will include visualizing fraud trends, uncovering hidden relationships, and establishing baseline statistics.

### 5. Actionable Insights & Final Presentation
- **Objective:** By the end of the project, we aim to deliver a comprehensive set of insights that can inform decision-making regarding fraud prevention and detection strategies. These findings will be shared through a final presentation and a dedicated project website hosted via GitHub Pages.


In [ ]:
#!pip install geopandas

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
import requests
from io import StringIO
warnings.simplefilter(action="ignore")
import requests
from bs4 import BeautifulSoup
import json
from pprint import pprint
from scipy.stats import linregress

In [ ]:
data_twelve_month = pd.read_csv("https://raw.githubusercontent.com/ryantangmj/ryantangmj.github.io/refs/heads/main/cargo_fraud_only.csv", index_col=0)
data_twelve_month = data_twelve_month.reset_index()
data_twelve_month.head()

In [ ]:
data_cargo_fraud = data_twelve_month[data_twelve_month["offense_name"].isin(["Credit Card/Automated Teller Machine Fraud", 
                                              "Wire Fraud", 
                                              "Welfare Fraud"])]
data_cargo_fraud.head()


In [ ]:
# calculate the proportion of fraud by region
fraud_by_state = data_cargo_fraud['region_name'].value_counts(normalize=True).sort_values(ascending=False)

# plot the bar chart
plt.figure(figsize=(12, 8))
plt.bar(fraud_by_state.index, fraud_by_state.values, color='skyblue')
plt.title('Proportion of Fraud Occurrences by Region', fontsize=16)
plt.xlabel('State', fontsize=12)
plt.ylabel('Proportion of Fraud Occurrences', fontsize=12) 
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


In [ ]:
# calculate the proportion of fraud by state
fraud_by_state = data_cargo_fraud['state_name'].value_counts(normalize=True).sort_values(ascending=False)

# retrieving geopandas usa data
usa = gpd.read_file('https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_state_500k.zip')

# merge the geopandas file and fraud data
merged = usa.merge(fraud_by_state, how='left', left_on='NAME', right_on='state_name')
merged['proportion'] = merged['proportion'].fillna(0)

# exclude Alaska and Hawaii before reprojection
merged = merged[~merged['STUSPS'].isin(['AK', 'HI'])]

# reproject the GeoDataFrame
visframe = merged.to_crs(epsg=2163)

# plot the map
fig, ax = plt.subplots(1, 1, figsize=(20, 12))
visframe.plot(column='proportion',
              cmap='OrRd',
              linewidth=0.8,
              ax=ax,
              edgecolor='0.8',
              legend=True,
              legend_kwds={
                  'label': "Proportion of Fraud Occurrences by State",
                  'orientation': "horizontal"
              })

ax.set_xlim([-2.5e6, 2.5e6])
ax.set_ylim([-2.5e6, 1.5e6])

ax.set_axis_off()
ax.set_title('Proportion of Fraud Occurrences by State', fontdict={'fontsize': 20}, pad=20)
plt.tight_layout()
plt.show()

In [ ]:
# calculate the proportion of fraud by state
fraud_by_state = data_cargo_fraud['location_name'].value_counts(normalize=True).sort_values(ascending=False)

# plot the bar chart
plt.figure(figsize=(12, 8))
plt.bar(fraud_by_state.index, fraud_by_state.values, color='skyblue')
plt.title('Proportion of Fraud Occurrences by Location', fontsize=16)
plt.xlabel('Location', fontsize=12)
plt.ylabel('Proportion of Fraud Occurrences', fontsize=12)
plt.xticks(rotation=45, fontsize=10, ha='right')  
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# calculate the proportion of fraud by state
fraud_by_state = data_cargo_fraud['offender_race'].value_counts(normalize=True).sort_values(ascending=False)

# plot the bar chart
plt.figure(figsize=(12, 8))
plt.bar(fraud_by_state.index, fraud_by_state.values, color='skyblue')
plt.title('Proportion of Fraud Occurrences by Race', fontsize=16)
plt.xlabel('Race', fontsize=12)
plt.ylabel('Proportion of Fraud Occurrences', fontsize=12)
plt.xticks(rotation=45, fontsize=10, ha='right')  
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# count the occurrences of fraud by victim type
fraud_by_type = data_cargo_fraud['victim_type_name'].value_counts(normalize=True).sort_values(ascending=False)

# plotting the bar plot
plt.figure(figsize=(10, 6))
plt.bar(fraud_by_type.index, fraud_by_type.values, color='skyblue')
plt.xlabel('Victim Type')
plt.xticks(rotation=45)
plt.ylabel('Number of Fraudulent Transactions')
plt.title('Proportion of Fraudulent Transactions by Victim Type')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
# count the occurrences of fraud by year
fraud_by_year = data_cargo_fraud['data_year'].value_counts().sort_index()

# plotting the line graph
plt.figure(figsize=(10, 6))
plt.plot(fraud_by_year.index, fraud_by_year.values, color='skyblue', marker='o')
plt.xlabel('Year')
plt.xticks(rotation=45)
plt.ylabel('Number of Fraudulent Transactions')
plt.title('Fraudulent Transactions by Year')
plt.tight_layout() 
plt.show()
plt.show()

In [ ]:
data_arrest = pd.read_csv("https://raw.githubusercontent.com/ryantangmj/ryantangmj.github.io/refs/heads/main/fraud_onemonth_data.csv", index_col=0)
data_arrest = data_arrest.reset_index()
data_arrest.head()

In [ ]:
# converting State Code column to str type
data_arrest['State Code'] = data_arrest['State Code'].astype(str).str.strip()

# dictionary of state codes to state
state_codes = {
    '50': 'Alaska',
    '01': 'Alabama',
    '03': 'Arkansas',
    '54': 'American Samoa',
    '02': 'Arizona',
    '04': 'California',
    '05': 'Colorado',
    '06': 'Connecticut',
    '52': 'Canal Zone',
    '08': 'District of Columbia',
    '07': 'Delaware',
    '09': 'Florida',
    '10': 'Georgia',
    '55': 'Guam',
    '51': 'Hawaii',
    '14': 'Iowa',
    '11': 'Idaho',
    '12': 'Illinois',
    '13': 'Indiana',
    '15': 'Kansas',
    '16': 'Kentucky',
    '17': 'Louisiana',
    '20': 'Massachusetts',
    '19': 'Maryland',
    '18': 'Maine',
    '21': 'Michigan',
    '22': 'Minnesota',
    '24': 'Missouri',
    '23': 'Mississippi',
    '25': 'Montana',
    '26': 'Nebraska',
    '32': 'North Carolina',
    '33': 'North Dakota',
    '28': 'New Hampshire',
    '29': 'New Jersey',
    '30': 'New Mexico',
    '27': 'Nevada',
    '31': 'New York',
    '34': 'Ohio',
    '35': 'Oklahoma',
    '36': 'Oregon',
    '37': 'Pennsylvania',
    '53': 'Puerto Rico',
    '38': 'Rhode Island',
    '39': 'South Carolina',
    '40': 'South Dakota',
    '41': 'Tennessee',
    '42': 'Texas',
    '43': 'Utah',
    '62': 'Virgin Islands',
    '45': 'Virginia',
    '44': 'Vermont',
    '46': 'Washington',
    '48': 'Wisconsin',
    '47': 'West Virginia',
    '49': 'Wyoming'
}

# map the state codes to state names
data_arrest['State'] = data_arrest['State Code'].map(state_codes)
data_arrest.head()

In [ ]:
# calculate the proportion of fraud by state
fraud_by_state = data_arrest['State'].value_counts(normalize=True).sort_values(ascending=False)

# retrieving geopandas usa data
usa = gpd.read_file('https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_state_500k.zip')

# merge the geopandas file and fraud data
merged = usa.merge(fraud_by_state, how='left', left_on='NAME', right_on='State')
merged['proportion'] = merged['proportion'].fillna(0)

# exclude Alaska and Hawaii before reprojection
merged = merged[~merged['STUSPS'].isin(['AK', 'HI'])]

# reproject the GeoDataFrame
visframe = merged.to_crs(epsg=2163)

# plot the map
fig, ax = plt.subplots(1, 1, figsize=(20, 12))
visframe.plot(column='proportion',
              cmap='OrRd',
              linewidth=0.8,
              ax=ax,
              edgecolor='0.8',
              legend=True,
              legend_kwds={
                  'label': "Proportion of Fraud Occurrences by State",
                  'orientation': "horizontal"
              })

ax.set_xlim([-2.5e6, 2.5e6])
ax.set_ylim([-2.5e6, 1.5e6])

ax.set_axis_off()
ax.set_title('Proportion of Fraud Occurrences by State', fontdict={'fontsize': 20}, pad=20)
plt.tight_layout()
plt.show()

In [ ]:
# calculate the occurrences of fraud by race
fraud_adult_white = data_arrest['Adult White'].sum()
fraud_adult_black = data_arrest['Adult Black'].sum()
fraud_adult_hispanic = data_arrest['Adult Hispanic'].sum()
fraud_adult_indian = data_arrest['Adult Indian'].sum()
fraud_adult_asian = data_arrest['Adult Asian'].sum()


# combine the counts into a single Series
fraud_by_race = pd.Series({
    'White': fraud_adult_white,
    'Black': fraud_adult_black,
    'Hispanic': fraud_adult_hispanic,
    'Indian': fraud_adult_indian,
    'Asian': fraud_adult_asian
})

# plot the bar chart
plt.figure(figsize=(12, 8))
plt.bar(fraud_by_race.index, fraud_by_race.values, color='skyblue')
plt.title('Proportion of Fraud Occurrences by Race', fontsize=16)
plt.xlabel('Race', fontsize=12)
plt.ylabel('Number of Fraud Occurrences', fontsize=12)
plt.xticks(rotation=45, fontsize=10, ha='right')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

**Scraping U.S. Bureau of Labor Statistics Website for Unemployment Rate data**

Visualizing the response received from scraping

In [ ]:
# define the payload data
payload = {
    "survey": "la",
    "map": "state",
    "seasonal": "s",
    "datatype": "unemployment",
    "year": "2012",
    "period": "M12"
}

# send the POST request
response = requests.post("https://data.bls.gov/lausmap/drawMap", data=payload)

# check if the request was successful
if response.status_code == 200:
    # parse the JSON response
    data = response.json()

    # extract the datatableList part
    table_data = data.get('tableData')
    datatable_list = table_data.get('datatableList')

    # print the datatableList
    pprint(datatable_list)
else:
    print(f"Failed to retrieve data. Status code: {response.status_code}")

In [ ]:
# initialise an empty dictionary to store data
data_dict = {}

# loop over the years from 2012 to 2022
for year in range(2012, 2023):
    # define the payload data for the endpoint
    payload = {
        "survey": "la",
        "map": "state",
        "seasonal": "s",
        "datatype": "unemployment",
        "year": str(year),
        "period": "M12"
    }

    # send the POST request
    response = requests.post("https://data.bls.gov/lausmap/drawMap", data=payload)

    # check if the request was successful
    if response.status_code == 200:
        # parse the JSON response
        data = response.json()

        # extract the datatableList part
        table_data = data.get('tableData')
        datatable_list = table_data.get('datatableList')

        # store the data in the dictionary
        for entry in datatable_list:
            region = entry['displayRegion']
            value = entry['currentData']
            if region not in data_dict:
                data_dict[region] = {}
            data_dict[region][year] = value
    else:
        print(f"Failed to retrieve data for year {year}. Status code: {response.status_code}")

# convert the dictionary to a DataFrame
df_unemployment_intermediate = pd.DataFrame(data_dict).T

# reset index to convert the DataFrame to a long format
df_unemployment_intermediate.reset_index(inplace=True)

# rename the index column to 'state'
df_unemployment_intermediate.rename(columns={'index': 'state'}, inplace=True)

# melt the DataFrame to long format
df_unemployment = pd.melt(df_unemployment_intermediate, id_vars=['state'], var_name='year', value_name='unemployment_rate')

# convert the 'year' and 'unemployment_rate' column to integer
df_unemployment['year'] = df_unemployment['year'].astype(int)
df_unemployment['unemployment_rate'] = df_unemployment['unemployment_rate'].astype(float)

# display the reshaped DataFrame
display(df_unemployment)

In [ ]:
# calculate the average unemployment rate by year
average_unemployment_by_year = df_unemployment.groupby('year')['unemployment_rate'].mean()

# plot the line graph
plt.figure(figsize=(10, 6))
plt.plot(average_unemployment_by_year.index, average_unemployment_by_year.values, color='skyblue', marker='o')
plt.xlabel('Year')
plt.xticks(rotation=45)
plt.ylabel('Average Unemployment Rate')
plt.title('Average Unemployment Rate by Year')
plt.tight_layout()
plt.show()

In [ ]:
# filtering data for the year 2022
df_unemployment_2022 = df_unemployment[df_unemployment['year'] == 2022]

# retrieving geopandas usa data
usa = gpd.read_file('https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_state_500k.zip')

# merge the geopandas file and unemployment data
merged = usa.merge(df_unemployment_2022, how='left', left_on='NAME', right_on='state')
merged['unemployment_rate'] = merged['unemployment_rate'].fillna(0)

# exclude Alaska and Hawaii before reprojection
merged = merged[~merged['STUSPS'].isin(['AK', 'HI'])]

# reproject the GeoDataFrame
visframe = merged.to_crs(epsg=2163)

# plot the map
fig, ax = plt.subplots(1, 1, figsize=(20, 12))
visframe.plot(column='unemployment_rate',
              cmap='OrRd',
              linewidth=0.8,
              ax=ax,
              edgecolor='0.8',
              legend=True,
              legend_kwds={
                  'label': "Unemployment Rate by State in 2022",
                  'orientation': "horizontal"
              })

ax.set_xlim([-2.5e6, 2.5e6])
ax.set_ylim([-2.5e6, 1.5e6])

ax.set_axis_off()
ax.set_title('Unemployment Rate by State in 2022', fontdict={'fontsize': 20}, pad=20)
plt.tight_layout()
plt.show()

In [ ]:
# join on 'state' from df_unemployment and 'state_name' from data_cargo
# join on 'year' from df_unemployment and 'data_year' from data_cargo
merged_df = pd.merge(df_unemployment, data_twelve_month, left_on=['state', 'year'], right_on=['state_name', 'data_year'])

merged_df.drop(["data_year", "state_name"], axis=1, inplace=True)

# display the merged DataFrame
display(merged_df.head(10))

In [ ]:
# aggregate data_cargo to count the number of entries per state and year
cargo_counts = merged_df.groupby(['state', 'year']).agg(
    cargo_count=('state', 'size'),  
    unemployment_rate=('unemployment_rate', 'mean')
).reset_index()

# calculate Q1 (25th percentile) and Q3 (75th percentile)
Q1 = cargo_counts['cargo_count'].quantile(0.25)
Q3 = cargo_counts['cargo_count'].quantile(0.75)

# calculate the Interquartile Range (IQR)
IQR = Q3 - Q1

# define the outlier boundaries
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# filter out any outliers
cargo_counts_filtered = cargo_counts[(cargo_counts['cargo_count'] >= lower_bound) & (cargo_counts['cargo_count'] <= upper_bound)]

# plot the correlation
plt.figure(figsize=(12, 8))
sns.scatterplot(data=cargo_counts_filtered, x='unemployment_rate', y='cargo_count', hue='year', palette='tab10', alpha=0.7)
plt.xlabel('Unemployment Rate')
plt.ylabel('Number of Fraudulent Transactions')
plt.title('Correlation between Unemployment Rate and Fraudulent Transactions')
plt.legend(title='Year', bbox_to_anchor=(1.05, 1), loc='upper left')

slope, intercept, r_value, p_value, std_err = linregress(cargo_counts_filtered['unemployment_rate'], cargo_counts_filtered['cargo_count']) 
# create x values for the line
x_vals = np.array(cargo_counts_filtered['unemployment_rate'])
x_vals_sorted = np.sort(x_vals)
y_vals = slope * x_vals_sorted + intercept
# plot the regression line
plt.plot(x_vals_sorted, y_vals, color='red')

plt.tight_layout()
plt.show()

In [ ]:
# discretize the year into periods based on unique years
cargo_counts['period'] = pd.cut(cargo_counts['year'], 
                                bins=[2012, 2013, 2014, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023], 
                                labels=['2012', '2013', '2014', '2016', '2017', '2018', '2019', '2020', '2021', '2022'], right=False)

# plotting
fig, axes = plt.subplots(2, 5, figsize=(40, 10))

period_labels = ['2012', '2013', '2014', '2016', '2017', '2018', '2019', '2020', '2021', '2022']

# Flatten the axes array for easy iteration
axes = axes.flatten()

for ax, period in zip(axes, period_labels):
    data_period = cargo_counts[cargo_counts['period'] == period]
    # calculate Q1 (25th percentile) and Q3 (75th percentile)
    Q1 = data_period['cargo_count'].quantile(0.25)
    Q3 = data_period['cargo_count'].quantile(0.75)

    # calculate the Interquartile Range (IQR)
    IQR = Q3 - Q1

    # define the outlier boundaries
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # filter out any outliers
    data_period = data_period[(data_period['cargo_count'] >= lower_bound) & (data_period['cargo_count'] <= upper_bound)]
    sns.scatterplot(x='unemployment_rate', y='cargo_count', data=data_period, ax=ax)

    data_period = data_period.dropna(subset=['unemployment_rate', 'cargo_count'])
    slope, intercept, r_value, p_value, std_err = linregress(data_period['unemployment_rate'], data_period['cargo_count']) 
    # create x values for the line
    x_vals = np.array(data_period['unemployment_rate'])
    x_vals_sorted = np.sort(x_vals)
    y_vals = slope * x_vals_sorted + intercept
    # plot the regression line
    ax.plot(x_vals_sorted, y_vals, color='red')
    
    ax.set_title(f'Cargo Count vs. Unemployment Rate\n{period}')
    ax.set_xlabel('Unemployment Rate')
    ax.set_ylabel('Cargo Count')
    ax.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# Load cargo fraud data
data_twelve_month = pd.read_csv(
    "https://raw.githubusercontent.com/ryantangmj/ryantangmj.github.io/refs/heads/main/cargo_fraud_only.csv",
    index_col=0
)
data_twelve_month = data_twelve_month.reset_index()

# Filter for specific fraud offenses
data_cargo_fraud = data_twelve_month[data_twelve_month["offense_name"].isin([
    "Credit Card/Automated Teller Machine Fraud", 
    "Wire Fraud", 
    "Welfare Fraud"
])]

# Count fraudulent transactions by year
fraud_by_year = data_cargo_fraud['data_year'].value_counts().sort_index()

# Plot fraudulent transactions over time
plt.figure(figsize=(10, 6))
plt.plot(fraud_by_year.index, fraud_by_year.values, marker='o', linestyle='-')
plt.title('Fraudulent Transactions by Year')
plt.xlabel('Year')
plt.ylabel('Fraudulent Transactions: Wire Fraud, Welfare Fraud')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
from fredapi import Fred

# Replace 'YOUR_API_KEY' with your actual FRED API key
fred = Fred(api_key='9ba8dec145955e0a72bb984873129630')

# Fetch CPI data (CPIAUCSL: Consumer Price Index for All Urban Consumers)
cpi_data = fred.get_series('CPIAUCSL', observation_start='2008-01-01', observation_end='2022-12-31')

# Resample to annual frequency by taking the average
cpi_annual = cpi_data.resample('A').mean()

# Prepare CPI DataFrame
df_cpi = cpi_annual.reset_index(name='CPI')
df_cpi['year'] = df_cpi['index'].dt.year
df_cpi = df_cpi[['year', 'CPI']]

# Display the CPI data
display(df_cpi)

In [ ]:
fred = Fred(api_key='9ba8dec145955e0a72bb984873129630')

# Fetch the unemployment rate data (UNRATE: Unemployment Rate)
unemployment_data = fred.get_series('UNRATE', observation_start='2008-01-01', observation_end='2022-12-31')

# Resample to annual frequency by taking the average
unemployment_annual = unemployment_data.resample('A').mean()

# Prepare Unemployment DataFrame
df_unemployment = unemployment_annual.reset_index(name='Unemployment_Rate')
df_unemployment['year'] = df_unemployment['index'].dt.year
df_unemployment = df_unemployment[['year', 'Unemployment_Rate']]

# Display the unemployment data
display(df_unemployment)



In [ ]:
# Fetch the housing price index data (CSUSHPINSA)
housing_data = fred.get_series('CSUSHPINSA', observation_start='2008-01-01', observation_end='2022-12-31')

# Resample to annual frequency by taking the average
housing_annual = housing_data.resample('A').mean()

# Prepare Housing DataFrame
df_housing = housing_annual.reset_index(name='Housing_Price_Index')
df_housing['year'] = df_housing['index'].dt.year
df_housing = df_housing[['year', 'Housing_Price_Index']]

# Display the housing price data
display(df_housing)

In [ ]:
# Aggregate fraudulent transactions by year
fraud_by_year = data_cargo_fraud.groupby('data_year').size().reset_index(name='Fraud_Count')
fraud_by_year.rename(columns={'data_year': 'year'}, inplace=True)

# Merge the datasets
merged_data = fraud_by_year.merge(df_cpi, on='year', how='left')
merged_data = merged_data.merge(df_unemployment, on='year', how='left')
merged_data = merged_data.merge(df_housing, on='year', how='left')

# Display the merged data
display(merged_data)


In [ ]:
plt.figure(figsize=(12, 6))
sns.lineplot(data=merged_data, x='year', y='Fraud_Count', marker='o')
plt.title('Number of Fraudulent Transactions Over Time')
plt.xlabel('Year')
plt.ylabel('Fraudulent Transactions')
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(12, 6))
sns.lineplot(data=merged_data, x='year', y='Unemployment_Rate', marker='o', color='green')
plt.title('Unemployment Rate Over Time')
plt.xlabel('Year')
plt.ylabel('Unemployment Rate (%)')
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# Ensure there are no missing values
merged_data_clean = merged_data.dropna(subset=['Fraud_Count', 'CPI', 'Unemployment_Rate', 'Housing_Price_Index'])

# Calculate correlation matrix
corr_matrix = merged_data_clean[['Fraud_Count', 'CPI', 'Unemployment_Rate', 'Housing_Price_Index']].corr()
print(corr_matrix)


In [ ]:
plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix Heatmap')
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(8, 6))
sns.scatterplot(data=merged_data, x='Unemployment_Rate', y='Fraud_Count', color='green')
sns.regplot(data=merged_data, x='Unemployment_Rate', y='Fraud_Count', scatter=False, color='red')
plt.title('Cargo Fraudulent Transactions vs. Unemployment Rate')
plt.xlabel('Unemployment Rate (%)')
plt.ylabel('Fraudulent Transactions')
plt.grid(True)
plt.tight_layout()
plt.show()


**FBI Crime Data** :
1. The information here is from the FBI Crime Data Explorer


In [ ]:
import requests

# Search for datasets related to 'fraud'
url = 'https://catalog.data.gov/api/3/action/package_search'
params = {
    'q': 'fraud',
    'rows': 10  # Number of results to return
}

response = requests.get(url, params=params)
if response.status_code == 200:
    data = response.json()
    results = data['result']['results']
    for dataset in results:
        print(f"Title: {dataset['title']}")
        resource_url = dataset['resources'][0]['url'] if dataset['resources'] else 'No URL available'
        print(f"URL: {resource_url}\n")
else:
    print(f"Failed to retrieve datasets: {response.status_code}")
    print(response.text)


In [ ]:
import requests
import pandas as pd

# Base API endpoint
url = 'https://www.consumerfinance.gov/data-research/consumer-complaints/search/api/v1/'

# Define the parameters for the API request
params = {
    'date_received_min': '2011-12-01',  # Data available from December 2011 onwards
    'date_received_max': '2022-12-31',
    'issue': ['Identity theft', 'Fraud or scam', 'Unauthorized transactions or other transaction problem'],
    'field': [
        'date_received',
        'product',
        'issue',
        'company',
        'state',
        'zip_code',
        'company_response',
  # Corrected field name
        # Removed 'consumer_disputed'
    ],
    'size': 1000,  # Number of records per page (maximum is 1000)
    'no_aggs': True,
    'page': 1  # Start with page 1
}

# Initialize an empty DataFrame to store the data
fraud_data = pd.DataFrame()

# Loop to handle pagination
while True:
    print(f"Retrieving page {params['page']}")
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        hits = data.get('hits', {}).get('hits', [])
        if not hits:
            break  # No more data
        # Extract the data
        records = [hit['_source'] for hit in hits]
        df_page = pd.DataFrame(records)
        fraud_data = pd.concat([fraud_data, df_page], ignore_index=True)
        # Move to the next page
        params['page'] += 1
    else:
        print(f"Failed to retrieve data: {response.status_code}")
        print(response.text)
        break

# Display the collected data
print(f"Total records retrieved: {len(fraud_data)}")
print(fraud_data.head())


In [ ]:
hpi_data = pd.read_csv("https://raw.githubusercontent.com/ryantangmj/ryantangmj.github.io/refs/heads/main/hpi_by_state.csv")
hpi_data.head()

In [ ]:
hpi_data.drop(columns=["Abbreviation", "FIPS", "Annual Change (%)", "HPI with 1990 base", "HPI with 2000 base"], inplace=True)
hpi_data = hpi_data[(hpi_data["Year"] >= 2012) &  (hpi_data["Year"] <= 2022)]
hpi_data.reset_index(drop=True, inplace=True)
hpi_data.head()

In [ ]:
hpi_by_year = hpi_data.groupby("Year").agg(HPI=('HPI', 'mean')
).reset_index()

# Plot HPI over time
plt.figure(figsize=(10, 6))
plt.plot(hpi_by_year.Year, hpi_by_year.HPI, marker='o', linestyle='-')
plt.title('Housing Price Index by Year')
plt.xlabel('Year')
plt.ylabel('Housing Price Index')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
hpi_by_state = hpi_data.groupby("State").agg(HPI=('HPI', 'median')
).reset_index()

# retrieving geopandas usa data
usa = gpd.read_file('https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_state_500k.zip')

# merge the geopandas file and unemployment data
merged = usa.merge(hpi_by_state, how='left', left_on='NAME', right_on='State')

# exclude Alaska and Hawaii before reprojection
merged = merged[~merged['STUSPS'].isin(['AK', 'HI'])]

# reproject the GeoDataFrame
visframe = merged.to_crs(epsg=2163)

# plot the map
fig, ax = plt.subplots(1, 1, figsize=(20, 12))
visframe.plot(column='HPI',
              cmap='OrRd',
              linewidth=0.8,
              ax=ax,
              edgecolor='0.8',
              legend=True,
              legend_kwds={
                  'label': "Median Housing Price Index by State from 2012 to 2022",
                  'orientation': "horizontal"
              })

ax.set_xlim([-2.5e6, 2.5e6])
ax.set_ylim([-2.5e6, 1.5e6])

ax.set_axis_off()
ax.set_title('Median Housing Price Index by State from 2012 to 2022', fontdict={'fontsize': 20}, pad=20)
plt.tight_layout()
plt.show()

In [ ]:
# join on 'State' from df_unemployment and 'state_name' from data_cargo
# join on 'Year' from df_unemployment and 'data_year' from data_cargo
merge_hpi_fraud_df = pd.merge(hpi_data, data_cargo_fraud, left_on=['State', 'Year'], right_on=['state_name', 'data_year'])

merge_hpi_fraud_df.drop(["data_year", "state_name"], axis=1, inplace=True)

# display the merged DataFrame
display(merge_hpi_fraud_df.head(10))

In [ ]:
# Aggregate fraudulent transactions by year
fraud_by_year_state = merge_hpi_fraud_df.groupby(['Year', 'State', 'HPI']).size().reset_index(name='Fraud_Count')

plt.figure(figsize=(10, 6))
plt.scatter(fraud_by_year_state['HPI'], fraud_by_year_state['Fraud_Count'])
plt.title('Scatter Plot of HPI vs Fraud Count')
plt.xlabel('Housing Price Index')
plt.ylabel('Fraud Count')
plt.grid(True)
plt.show()
